<a href="https://colab.research.google.com/github/SChoi005/DeepLearningBasics/blob/main/distributed_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distributed Training

## Installing dependencies and setting up the environment

In [ ]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.8.2+zzzcolab20220527125636
Uninstalling tensorflow-2.8.2+zzzcolab20220527125636:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.2+zzzcolab20220527125636.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220527125636


In [ ]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.3 kB/s 
     |████████████████████████████████| 5.8 MB 1.2 MB/s 
     |████████████████████████████████| 1.6 MB 25.2 MB/s 
     |████████████████████████████████| 438 kB 58.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5

In [3]:
import time
import numpy as np
import tensorflow as tf
tf.__version__

'2.9.1'

## Dataset preprocessing

In [5]:
# Loading the MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [7]:
X_train.shape

(60000, 28, 28)

In [9]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [10]:
X_train.shape

(60000, 784)

## Distributed Training

In [11]:
# Defining normal (non distributed) model
model_normal = tf.keras.models.Sequential()

In [13]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))

In [14]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [15]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [16]:
model_normal.compile(optimizer='Adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['sparse_categorical_crossentropy'])

## Defining the Distributed Strategy

In [17]:
distribute =  tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [19]:
# Definning a distributed model
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='Adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['sparse_categorical_crossentropy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


## Speed comparison between normal training and distributed training process

In [20]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("Distributed training took: {}".format(time.time() - start_time))

Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2400/2400 [==============================] - 12s 3ms/step - loss: 0.2813 - sparse_categorical_crossentropy: 0.2813
Epoch 2/10
2400/2400 [============

In [21]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("Normal training took: {}".format(time.time() - start_time))

Epoch 1/10
2400/2400 [==============================] - 6s 2ms/step - loss: 0.2817 - sparse_categorical_crossentropy: 0.2817
Epoch 2/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.1372 - sparse_categorical_crossentropy: 0.1372
Epoch 3/10
2400/2400 [==============================] - 6s 2ms/step - loss: 0.1034 - sparse_categorical_crossentropy: 0.1034
Epoch 4/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0852 - sparse_categorical_crossentropy: 0.0852
Epoch 5/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0738 - sparse_categorical_crossentropy: 0.0738
Epoch 6/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0632 - sparse_categorical_crossentropy: 0.0632
Epoch 7/10
2400/2400 [==============================] - 6s 2ms/step - loss: 0.0570 - sparse_categorical_crossentropy: 0.0570
Epoch 8/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.0522 - sparse_categorical_crossentropy: 0.0522
